In [ ]:
import numpy as np
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7_NORM

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

In [ ]:
num_im = 100
num_ta = 100
num_sols = 1

solver.shrink_ratio = 0.0
J_im, P_im, F_im = solver.random_sample_JPF(num_samples=num_im)
J_ta, P_ta, F_ta = solver.random_sample_JPF(num_samples=num_ta)

In [ ]:
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, (j_im, f_im) in enumerate(zip(J_im, F_im)):
    F = np.full_like(F_ta, *f_im)
    J = solver.solve(P=P_ta, F=F, num_sols=num_sols, return_numpy=True)
    l2_errs[i], ang_errs[i] = solver.evaluate_solutions(J, P_ta, return_col=True)
    config_errs[i] = np.linalg.norm(J - j_im, axis=2).mean(axis=0)

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()

In [ ]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

# -> unrefined solutions
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, p in enumerate(P_ta):
    (
        solutions,
        l2_errors,
        angular_errors,
        _,
        _,
        _,
    ) = ik_solver.solve(
        p, n=num_im, refine_solutions=False, return_detailed=True
    )  # type: ignore

    solutions = solutions.detach().cpu().numpy()

    l2_errs[:, i] = l2_errors
    ang_errs[:, i] = angular_errors

    for j, j_im in enumerate(J_im):
        config_errs[j, i] = np.linalg.norm(solutions - j_im, axis=1).mean()

In [ ]:
df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()